In [149]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report

import pandas as pd
import numpy as np
import os
import gc

ModuleNotFoundError: No module named 'tensorflow'

In [122]:
batch_size = 256
lr = 0.0001
epochs = 200

cpu


In [146]:
dataset0_classes = os.listdir('dataset0/train/')
dataset0_label_encoder = LabelEncoder()

dataset0_label_encoder.fit(dataset0_classes)
print(dataset0_label_encoder.classes_)

dataset1_classes = os.listdir('dataset1/train/')
dataset1_label_encoder = LabelEncoder()

dataset1_label_encoder.fit(dataset1_classes)
print(dataset1_label_encoder.classes_)

['.DS_Store' 'class0' 'class1' 'class11' 'class12' 'class13' 'class14'
 'class15' 'class2' 'class3' 'class4' 'class5' 'class6' 'class7' 'class8'
 'class9']
['.DS_Store' 'class0' 'class1' 'class10' 'class2' 'class3' 'class4'
 'class5' 'class6' 'class7' 'class8' 'class9']


In [147]:
dataset0_dir = 'dataset0/'
dataset1_dir = 'dataset1/'

dataset0_train_dir = dataset0_dir + 'train/'
dataset0_test_dir = dataset0_dir + 'test/'

dataset1_train_dir = dataset1_dir + 'train/'
dataset1_test_dir = dataset1_dir + 'test/'

In [154]:
def create_dataset(dataset_dir):
    X, y = [], []
    labels = os.listdir(dataset_dir)
    if '.DS_Store' in labels:
        labels.remove('.DS_Store')
    for label in labels:
        file_list = os.listdir(dataset_dir + label + '/')
        for f in file_list:
            temp = pd.read_csv(dataset_dir + label + '/' + f)
            X.append(temp.values)
            y.append(label)
    
    return X, y

In [156]:
d0_x_train, d0_y_train = create_dataset(dataset0_train_dir)
d0_y_train = dataset0_label_encoder.transform(d0_y_train)

d0_x_test, d0_y_test = create_dataset(dataset0_test_dir)
d0_y_test = dataset0_label_encoder.transform(d0_y_test)

In [109]:
def Train(model, train_loader, val_loader):
    running_loss = .0
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr
    
    model.train()
    for idx, (inputs,labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(inputs.float())
        loss = criterion(preds.to(torch.float32), labels.to(torch.float32))
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss.detach().numpy())
    
    print(f'train_loss {train_loss}')

In [128]:
from torch.optim.lr_scheduler import _LRScheduler

class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [143]:
kargs = {}

class model(nn.Module):
    def __init__(self, drop=.3):
        super(model, self).__init__()
        conv1 = nn.Conv1d(1, 6, 5, 1) 
        
        # activation ReLU
        pool1 = nn.MaxPool1D(2)

        self.conv_module = nn.Sequential(
            conv1,
            nn.ReLU(),
            pool1
        )
        
        fc1 = nn.Linear(16*4*4, 120)
        # activation ReLU
        fc2 = nn.Linear(120, 84)
        # activation ReLU
        fc3 = nn.Linear(84, 10)

        self.fc_module = nn.Sequential(
            fc1,
            nn.ReLU(),
            fc2,
            nn.ReLU(),
            fc3
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        return x

model = model(**kargs).to(device)
print(model)

model(
  (rnn): LSTM(10, 256, num_layers=3, batch_first=True)
  (fc): Linear(in_features=256, out_features=6, bias=True)
)


In [110]:
train_losses = []
valid_losses = []

for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train(train_loader=dataset1_train_dataloader)
    gc.collect()

epochs 1/500


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x203520 and 512x32)

In [27]:
model.eval()
predicted = []

with torch.no_grad():
    for idx, x in enumerate(dataset1_test_dataloader):
        x = x.permute(0, 2, 1).contiguous().to(device)

        optimizer.zero_grad()
        output = model(x.float())

        _, preds = torch.max(output, 1)
        predicted.extend(preds.cpu().numpy())
    torch.cuda.empty_cache()

RuntimeError: Given groups=1, weight of size [6, 236, 6], expected input[207, 6, 800] to have 236 channels, but got 6 channels instead

In [ ]:
pd_preds = pd.DataFrame(predicted, columns=['predicted value'])
pd_preds.to_csv('submission.csv')
pd_preds.head()